# Code used for Training of the Agent

## Importing the neccesary libraries

In [1]:
# Import retro to play Street Fighter using a ROM
import retro
# Import time to slow down game
import time

##Environment Pre-proccesing Libraries

# Import environment base class for a wrapper 
from gym import Env 
# Import the space shapes for the environment (Multibinary gives us the action_space type and Box helps us return an array of any size)
from gym.spaces import MultiBinary, Box
# Import numpy to calculate frame delta 
import numpy as np
# Import opencv for grayscaling
import cv2
# Import matplotlib for plotting the image
from matplotlib import pyplot as plt

##Hyperparameter Tuning

# Importing optuna for HPO
import optuna
# Import PPO to implemnent PPO algo for RL
from stable_baselines3 import PPO
# Import evaluate_policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# Import base callback to load models
from stable_baselines3.common.callbacks import BaseCallback
# Import os to deal with filepaths
import os

### Global Variables

In [ ]:
# File path to log out the model data
LOG_DIR = './logs/'
# File path to save the optimized parameter values
OPT_DIR = './opt/'
#File path to save checkpoints
CHECKPOINT_DIR = './train/'

### Making the game environment and Pre-process the environment inside of a custom environment wrapper

In [ ]:
# Create custom environment which pre-processes the input frames
class StreetFighter(Env): 
    def __init__(self):
        super().__init__()
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        # Start the game 
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
        #retro.Actions.FILTERED filters the action space to give the VALID BUTTON COMBINATIONS and removes the irrelevant actions
    
    def reset(self):
        # Fetch the first frame 
        obs = self.game.reset()
        # Preprocess the frame
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create a variable to store the score delta 
        self.score = 0 
        
        # Return the preprocessed frame
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling (as only the positions are relevant)
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        
        # Resize (for faster training)
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        
        # Add the channels value (change the frame size to be (84,84,1) from (84,84))
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
     def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        # obs contains the current frame data
        obs = self.preprocess(obs) 
        
        # Change in character position
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Change the reward function to be the Score Delta
        reward = (self.enemy_health - info['enemy_health'])*2 + (info['health'] - self.health) 
        self.score = info['score'] 
        
        # Instead of the actual frame data, a pre-processed frame is returned along with the other attributes
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        # Render the game
        self.game.render()
        
    def close(self):
        # Close the game
        self.game.close()

### Hyperparameter Optimization/Tuning

In [ ]:
# Function to return test hyperparameters
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [ ]:
# Run a training loop and return mean reward
# Try-except loop is present to avoid premature termination of training due to invalid parameter
def optimize_agent(trial):
    try:
        # Store the suggested parameters
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = StreetFighter()
        env = Monitor(env, LOG_DIR) # allows us to log mean episode reward and mean episode length from the vectorized environment
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algorithm 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        # A large negative value ensures that the case that causes the error is definetely not selected (as the reward is positive)
        return -1000

In [ ]:
# Creating the experiment 
study = optuna.create_study(direction='maximize')

# Optimizing over the given number of trails, with each trial going through total_timesteps timesteps to get the mean reward for that trial
study.optimize(optimize_agent, n_trials=100, n_jobs=1)

In [2]:
# Gives the best set of parameters among all the trails done under the experiment study
study.best_params

NameError: name 'study' is not defined

In [ ]:
#Load the best set of parameters
model = PPO.load(os.path.join(OPT_DIR, 'trial_24_best_model.zip'))

### Callback Setup

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

## Training

In [ ]:
# Create environment 
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
model_params = study.best_params
model_params['n_steps'] = 7488  # set n_steps to 7488 or a factor of 64
model_params['learning_rate'] = 5e-7 # set the learning rate to be a small value
model_params

In [ ]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

In [ ]:
# Reload previous weights
model.load(os.path.join(OPT_DIR, 'trial_24_best_model.zip'))

In [ ]:
# Training the model 
model.learn(total_timestep=5000000, callback=callback) 

### Evaluate Model

In [ ]:
model = PPO.load('./opt/trial_24_best_model.zip')

In [ ]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

In [ ]:
mean_reward

## Model Testing

In [ ]:
obs = env.reset()

In [ ]:
env.step(model.predict(obs)[0])

In [ ]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        print(reward)